**1. Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:**

* InvoiceNo — номер транзакции
* StockCode — код товара
* Description — описание товара
* Quantity — количество единиц товара, добавленных в заказ
* InvoiceDate — дата транзакции 
* UnitPrice — цена за единицу товара
* CustomerID — id клиента
* Country — страна, где проживает клиент

**В данной задаче ссылка для считывания данных лежит в переменной path_to_file.
Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.**

In [24]:
from functools import total_ordering

import pandas as pd
from numpy.ma.extras import unique

patt_to_file = "data.csv"
retail = pd.read_csv(patt_to_file, encoding="ISO-8859-1")

In [25]:
retail_columns = retail.columns.tolist()
print(retail_columns)

['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


**2. Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.**

Вам понадобится len(название_колонки) и drop_duplicates()

In [26]:
duplicateCount = len(retail) - len(retail.drop_duplicates())
print(f"Количество дубликатов: {duplicateCount}")
retail = retail.drop_duplicates()
retail.head()

Количество дубликатов: 5268


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


**3. Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).**

**Сколько всего транзакций отменили пользователи? Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.**

Прежде чем начать, попробуйте метод retail.InvoiceNo.str[0]

Далее используйте метод .unique()

И сделайте выбоку по retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

In [27]:
canceledTransactionsCount = retail.loc[retail.InvoiceNo.str[0] == "C"]
print(canceledTransactionsCount)

       InvoiceNo StockCode                       Description  Quantity  \
141      C536379         D                          Discount        -1   
154      C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235      C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236      C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237      C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   
...          ...       ...                               ...       ...   
540449   C581490     23144   ZINC T-LIGHT HOLDER STARS SMALL       -11   
541541   C581499         M                            Manual        -1   
541715   C581568     21258        VICTORIAN SEWING BOX LARGE        -5   
541716   C581569     84978  HANGING HEART JAR T-LIGHT HOLDER        -1   
541717   C581569     20979     36 PENCILS TUBE RED RETROSPOT        -5   

            InvoiceDate  UnitPrice  CustomerID         Country  
141      12/1/2010 9:41      27.50     14527.0

In [28]:
uniqueCanceledProducts = canceledTransactionsCount["StockCode"].unique()

In [29]:
numberOfCanceledTransactions = len(uniqueCanceledProducts)
print(f"Количество отмененных транзакций: {numberOfCanceledTransactions}")

Количество отмененных транзакций: 1947


**5.Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).**

Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

**Подсказка:**
1. Начните с выборки retail.Country == 'Germany', группировки по 'CustomerID' и агрегации total_invoices=('InvoiceNo', 'nunique')
2. Далее из полученой выборки из предыдущего шага выподните метод: название_выборки.столбец.quantile(0.8)
3. Запишите полученные id пользователей в germany_top

In [32]:
germanyData = retail[retail.Country == "Germany"]
print(germanyData)

       InvoiceNo StockCode                          Description  Quantity  \
1109      536527     22809              SET OF 6 T-LIGHTS SANTA         6   
1110      536527     84347  ROTATING SILVER ANGELS T-LIGHT HLDR         6   
1111      536527     84945   MULTI COLOUR SILVER T-LIGHT HOLDER        12   
1112      536527     22242        5 HOOK HANGER MAGIC TOADSTOOL        12   
1113      536527     22244           3 HOOK HANGER MAGIC GARDEN        12   
...          ...       ...                                  ...       ...   
541801    581578     22993         SET OF 4 PANTRY JELLY MOULDS        12   
541802    581578     22907     PACK OF 20 NAPKINS PANTRY DESIGN        12   
541803    581578     22908        PACK OF 20 NAPKINS RED APPLES        12   
541804    581578     23215     JINGLE BELL HEART ANTIQUE SILVER        12   
541805    581578     22736          RIBBON REEL MAKING SNOWMEN         10   

            InvoiceDate  UnitPrice  CustomerID  Country  
1109    12/1/2010

In [33]:
customerOrders = germanyData.groupby("CustomerID").agg(totalInvoices=("InvoiceNo", "nunique")).reset_index()

In [37]:
nThreshold = customerOrders["totalInvoices"].quantile(0.8)
print(nThreshold)

9.0


In [36]:
germanyTop = customerOrders[customerOrders["totalInvoices"] >= nThreshold]["CustomerID"]
print(germanyTop)

3     12471.0
4     12472.0
5     12473.0
6     12474.0
8     12476.0
9     12477.0
12    12481.0
16    12500.0
37    12569.0
44    12600.0
52    12619.0
53    12621.0
57    12626.0
65    12647.0
70    12662.0
78    12705.0
79    12708.0
80    12709.0
82    12712.0
84    12720.0
Name: CustomerID, dtype: float64


**6. Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.**

Датафрейм записан в retail, нужные id пользователей – в germany_top.

**Подсказка**
Попробуйте использовать top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]

In [39]:
topRetailGermany = retail.loc[retail.CustomerID.isin(germanyTop)]
topRetailGermany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


**7. Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?** 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [48]:
filteredData = topRetailGermany[topRetailGermany.StockCode != "POST"]
productOrders = filteredData.groupby('StockCode').agg(total_orders=('InvoiceNo', 'nunique')).reset_index()

In [49]:
mostFrequentProducts = productOrders.loc[productOrders["total_orders"].idxmax()]
print(f"Продукт с кодом '{mostFrequentProducts['StockCode']}' добавляли в корзину {mostFrequentProducts['total_orders']} раз(а).")

Продукт с кодом '22326' добавляли в корзину 64 раз(а).


**8. Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.**

Отфильтрованные данные записаны в retail.

**Подсказка**
retail['новый_столбец'] = retail.старый_столбец_1 * retail.старый_столбец_2

In [50]:
retail["Revenue"] = retail["Quantity"] * retail["UnitPrice"]
print(retail[["Quantity", "UnitPrice", "Revenue"]].head())

   Quantity  UnitPrice  Revenue
0         6       2.55    15.30
1         6       3.39    20.34
2         8       2.75    22.00
3         6       3.39    20.34
4         6       3.39    20.34
